In [1]:
import tensorflow as tf
import os, warnings
import pandas as pd
warnings.filterwarnings('ignore')
import numpy as np
from logp.zq.io.smi2feat import load_feat_from_csv
from logp.zq.io.load_dataframe import load_pd
from logp.zq.model.model import potential_model
from sklearn.metrics import mean_absolute_error,mean_squared_error
from logp.zq.io.base import sparse_batch

In [2]:
filename = 'logp/DATASETS/sf_ave_PhysProp_MD_weights_charge.csv'
dataset = pd.read_csv(filename)
dataset.shape

(4802, 4)

In [3]:
results = load_pd(filename)

In [4]:
# load the predicting data in martel
filename_martel = 'logp/DATASETS/Martel/MD_ave/weight_q/sf_ave_Martel_MD_weights_charge.csv'
dataset_martel = pd.read_csv(filename_martel)
dataset_martel.shape
results_martel = load_pd(filename_martel)
dataset_martel = lambda: load_feat_from_csv(results_martel,24,split={'train':10,'test':0},shuffle=False)
martel_train = lambda: dataset_martel()['train'].apply(sparse_batch(5))

In [5]:
# load the predicting data in star
filename_star = 'logp/DATASETS/star_nonstar/MD_ave/weight_q/sf_ave_star_MD_weights_charge.csv'
dataset_star = pd.read_csv(filename_star)
dataset_star.shape
results_star = load_pd(filename_star)

dataset_star = lambda: load_feat_from_csv(results_star,24,split={'train':10,'test':0},shuffle=False)
star_train = lambda: dataset_star()['train'].apply(sparse_batch(5))

In [6]:
# load the predicting data in huusk
filename_nonstar = 'logp/DATASETS/Huuskonen/MD_ave/weight_q/sf_ave_Huuskonen_MD_weights_charge.csv'
dataset_nonstar = pd.read_csv(filename_nonstar)
dataset_nonstar.shape
results_nonstar = load_pd(filename_nonstar)

dataset_nonstar = lambda: load_feat_from_csv(results_nonstar,24,split={'train':10,'test':0},shuffle=False)
nonstar_train = lambda: dataset_nonstar()['train'].apply(sparse_batch(5))

In [7]:
logps_martel = []
logps_star = []
logps_nonstar = []

for x in range(10):
    # define the parameters
    params = {
    'model_dir':'MD_ave_weight_q_PhysProp/100_100_100/lr_1e-5/kfold_{}'.format(x),
    'network':'bpnn',
    'network_params':{
    'nn_spec':{1:[100,100,100],6:[100,100,100],7:[100,100,100],8:[100,100,100]}},
    'model_params': {'learning_rate': 1e-5}
    }
    #split the dataset into train and test
    ftrain = 'logp/DATASETS/PhysProps/MD_ave/weight_q/sf_ave_PhysProp_MD_weights_q_train_{}.csv'.format(x)
    ftest  = 'logp/DATASETS/PhysProps/MD_ave/weight_q/sf_ave_PhysProp_MD_weights_q_test_{}.csv'.format(x)
    dtrain = load_pd(ftrain)
    dtest = load_pd(ftest)
    dataset_train = lambda: load_feat_from_csv(dtrain,24,split={'train':10,'test':0})
    dataset_test = lambda: load_feat_from_csv(dtest,24,split={'train':0,'test':10})
    # yeild the train and test `input_fn`
    train = lambda: dataset_train()['train'].repeat().shuffle(1000).apply(sparse_batch(100))
    test = lambda: dataset_test()['test'].repeat().apply(sparse_batch(50))
    
    # initialize the model
    model = potential_model(params)
    # define the hooks
    early_stopping = tf.estimator.experimental.stop_if_no_decrease_hook(model, metric_name='loss',
                                                              max_steps_without_decrease=1000,
                                                              min_steps=100)
    # train and evaluate the model
    train_spec = tf.estimator.TrainSpec(input_fn=train, hooks=[early_stopping] ,max_steps=200000)
    eval_spec = tf.estimator.EvalSpec(input_fn=test, steps=400)
    tf.estimator.train_and_evaluate(model, train_spec, eval_spec)
    
    # predict the data in martel 
    preds_martel = model.predict(input_fn=martel_train)
    preds_star = model.predict(input_fn=star_train)
    preds_nonstar = model.predict(input_fn=nonstar_train)
    
    logp_martel = []
    for pred in preds_martel:
        logp_martel.append(pred['logp'])
    logps_martel.append(logp_martel)
    
    logp_star = []
    for pred in preds_star:
        logp_star.append(pred['logp'])
    logps_star.append(logp_star)
    
    logp_nonstar = []
    for pred in preds_nonstar:
        logp_nonstar.append(pred['logp'])
    logps_nonstar.append(logp_nonstar)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'MD_ave_weight_q_PhysProp/100_100_100/lr_1e-5/kfold_0', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Not using Distribute Coordina

INFO:tensorflow:global_step/sec: 103.254
INFO:tensorflow:loss = 0.3077795, step = 5400 (0.968 sec)
INFO:tensorflow:global_step/sec: 103.629
INFO:tensorflow:loss = 0.452798, step = 5500 (0.965 sec)
INFO:tensorflow:global_step/sec: 102.174
INFO:tensorflow:loss = 0.4447327, step = 5600 (0.979 sec)
INFO:tensorflow:global_step/sec: 103.44
INFO:tensorflow:loss = 0.37885436, step = 5700 (0.967 sec)
INFO:tensorflow:global_step/sec: 102.912
INFO:tensorflow:loss = 0.5091846, step = 5800 (0.972 sec)
INFO:tensorflow:global_step/sec: 101.737
INFO:tensorflow:loss = 0.28496024, step = 5900 (0.983 sec)
INFO:tensorflow:global_step/sec: 103.425
INFO:tensorflow:loss = 0.5830436, step = 6000 (0.966 sec)
INFO:tensorflow:global_step/sec: 103.29
INFO:tensorflow:loss = 0.28566825, step = 6100 (0.968 sec)
INFO:tensorflow:global_step/sec: 103.471
INFO:tensorflow:loss = 0.3875802, step = 6200 (0.967 sec)
INFO:tensorflow:global_step/sec: 102.725
INFO:tensorflow:loss = 0.3760271, step = 6300 (0.973 sec)
INFO:tenso

INFO:tensorflow:loss = 0.32454094, step = 13600 (0.973 sec)
INFO:tensorflow:global_step/sec: 102.972
INFO:tensorflow:loss = 0.28153172, step = 13700 (0.971 sec)
INFO:tensorflow:global_step/sec: 102.711
INFO:tensorflow:loss = 0.45322946, step = 13800 (0.974 sec)
INFO:tensorflow:global_step/sec: 102.671
INFO:tensorflow:loss = 0.352794, step = 13900 (0.974 sec)
INFO:tensorflow:global_step/sec: 102.278
INFO:tensorflow:loss = 0.32311118, step = 14000 (0.978 sec)
INFO:tensorflow:global_step/sec: 103.117
INFO:tensorflow:loss = 0.24186867, step = 14100 (0.969 sec)
INFO:tensorflow:global_step/sec: 103.13
INFO:tensorflow:loss = 0.2644668, step = 14200 (0.970 sec)
INFO:tensorflow:global_step/sec: 103.283
INFO:tensorflow:loss = 0.40011162, step = 14300 (0.968 sec)
INFO:tensorflow:global_step/sec: 102.634
INFO:tensorflow:loss = 0.34115937, step = 14400 (0.974 sec)
INFO:tensorflow:global_step/sec: 103.055
INFO:tensorflow:loss = 0.45872802, step = 14500 (0.970 sec)
INFO:tensorflow:global_step/sec: 10

INFO:tensorflow:global_step/sec: 102.482
INFO:tensorflow:loss = 0.27588606, step = 21800 (0.976 sec)
INFO:tensorflow:global_step/sec: 101.712
INFO:tensorflow:loss = 0.24698551, step = 21900 (0.983 sec)
INFO:tensorflow:global_step/sec: 103.454
INFO:tensorflow:loss = 0.24969964, step = 22000 (0.966 sec)
INFO:tensorflow:global_step/sec: 102.835
INFO:tensorflow:loss = 0.34627327, step = 22100 (0.972 sec)
INFO:tensorflow:global_step/sec: 103.09
INFO:tensorflow:loss = 0.30896783, step = 22200 (0.970 sec)
INFO:tensorflow:global_step/sec: 102.587
INFO:tensorflow:loss = 0.3361939, step = 22300 (0.975 sec)
INFO:tensorflow:global_step/sec: 101.945
INFO:tensorflow:loss = 0.30930233, step = 22400 (0.981 sec)
INFO:tensorflow:global_step/sec: 102.642
INFO:tensorflow:loss = 0.36903614, step = 22500 (0.974 sec)
INFO:tensorflow:global_step/sec: 103.199
INFO:tensorflow:loss = 0.4250949, step = 22600 (0.969 sec)
INFO:tensorflow:global_step/sec: 102.815
INFO:tensorflow:loss = 0.26217756, step = 22700 (0.97

INFO:tensorflow:global_step/sec: 103.439
INFO:tensorflow:loss = 0.32911175, step = 30000 (0.967 sec)
INFO:tensorflow:global_step/sec: 103.646
INFO:tensorflow:loss = 0.23036855, step = 30100 (0.965 sec)
INFO:tensorflow:global_step/sec: 102.341
INFO:tensorflow:loss = 0.44474244, step = 30200 (0.977 sec)
INFO:tensorflow:global_step/sec: 103.36
INFO:tensorflow:loss = 0.3010774, step = 30300 (0.968 sec)
INFO:tensorflow:global_step/sec: 102.634
INFO:tensorflow:loss = 0.37910283, step = 30400 (0.974 sec)
INFO:tensorflow:global_step/sec: 104.128
INFO:tensorflow:loss = 0.348253, step = 30500 (0.961 sec)
INFO:tensorflow:global_step/sec: 102.948
INFO:tensorflow:loss = 0.31265593, step = 30600 (0.971 sec)
INFO:tensorflow:global_step/sec: 102.731
INFO:tensorflow:loss = 0.2935498, step = 30700 (0.973 sec)
INFO:tensorflow:global_step/sec: 102.546
INFO:tensorflow:loss = 0.29365957, step = 30800 (0.975 sec)
INFO:tensorflow:global_step/sec: 103.258
INFO:tensorflow:loss = 0.27813494, step = 30900 (0.968 

...

INFO:tensorflow:global_step/sec: 105.152
INFO:tensorflow:loss = 0.19907635, step = 163900 (0.951 sec)
INFO:tensorflow:global_step/sec: 104.834
INFO:tensorflow:loss = 0.37675303, step = 164000 (0.953 sec)
INFO:tensorflow:global_step/sec: 105.311
INFO:tensorflow:loss = 0.3003514, step = 164100 (0.950 sec)
INFO:tensorflow:global_step/sec: 103.934
INFO:tensorflow:loss = 0.35441986, step = 164200 (0.962 sec)
INFO:tensorflow:global_step/sec: 103.291
INFO:tensorflow:loss = 0.18939693, step = 164300 (0.968 sec)
INFO:tensorflow:global_step/sec: 104.983
INFO:tensorflow:loss = 0.35915756, step = 164400 (0.952 sec)
INFO:tensorflow:global_step/sec: 104.946
INFO:tensorflow:loss = 0.2123439, step = 164500 (0.953 sec)
INFO:tensorflow:global_step/sec: 104.888
INFO:tensorflow:loss = 0.28711912, step = 164600 (0.953 sec)
INFO:tensorflow:global_step/sec: 104.087
INFO:tensorflow:loss = 0.3774294, step = 164700 (0.961 sec)
INFO:tensorflow:global_step/sec: 104.46
INFO:tensorflow:loss = 0.27859643, step = 164

INFO:tensorflow:global_step/sec: 105.339
INFO:tensorflow:loss = 0.39782032, step = 172000 (0.949 sec)
INFO:tensorflow:global_step/sec: 104.378
INFO:tensorflow:loss = 0.29937553, step = 172100 (0.958 sec)
INFO:tensorflow:global_step/sec: 104.07
INFO:tensorflow:loss = 0.3678844, step = 172200 (0.961 sec)
INFO:tensorflow:global_step/sec: 103.805
INFO:tensorflow:loss = 0.30234855, step = 172300 (0.964 sec)
INFO:tensorflow:global_step/sec: 104.503
INFO:tensorflow:loss = 0.17527962, step = 172400 (0.957 sec)
INFO:tensorflow:global_step/sec: 105.501
INFO:tensorflow:loss = 0.3431715, step = 172500 (0.948 sec)
INFO:tensorflow:global_step/sec: 104.835
INFO:tensorflow:loss = 0.33506408, step = 172600 (0.953 sec)
INFO:tensorflow:global_step/sec: 105.11
INFO:tensorflow:loss = 0.24108756, step = 172700 (0.951 sec)
INFO:tensorflow:global_step/sec: 105.021
INFO:tensorflow:loss = 0.30066046, step = 172800 (0.952 sec)
INFO:tensorflow:global_step/sec: 104.42
INFO:tensorflow:loss = 0.44026008, step = 1729

INFO:tensorflow:global_step/sec: 104.796
INFO:tensorflow:loss = 0.22053182, step = 180100 (0.954 sec)
INFO:tensorflow:global_step/sec: 104.425
INFO:tensorflow:loss = 0.2743171, step = 180200 (0.958 sec)
INFO:tensorflow:global_step/sec: 103.849
INFO:tensorflow:loss = 0.27575183, step = 180300 (0.963 sec)
INFO:tensorflow:global_step/sec: 103.949
INFO:tensorflow:loss = 0.34307012, step = 180400 (0.962 sec)
INFO:tensorflow:global_step/sec: 104.941
INFO:tensorflow:loss = 0.36160374, step = 180500 (0.953 sec)
INFO:tensorflow:global_step/sec: 104.825
INFO:tensorflow:loss = 0.2182948, step = 180600 (0.954 sec)
INFO:tensorflow:global_step/sec: 104.964
INFO:tensorflow:loss = 0.32701537, step = 180700 (0.953 sec)
INFO:tensorflow:global_step/sec: 105.643
INFO:tensorflow:loss = 0.3256568, step = 180800 (0.947 sec)
INFO:tensorflow:global_step/sec: 104.956
INFO:tensorflow:loss = 0.30197546, step = 180900 (0.953 sec)
INFO:tensorflow:global_step/sec: 104.94
INFO:tensorflow:loss = 0.189606, step = 18100

INFO:tensorflow:global_step/sec: 105.071
INFO:tensorflow:loss = 0.43643174, step = 188200 (0.952 sec)
INFO:tensorflow:global_step/sec: 104.121
INFO:tensorflow:loss = 0.23182638, step = 188300 (0.961 sec)
INFO:tensorflow:global_step/sec: 105.188
INFO:tensorflow:loss = 0.29861173, step = 188400 (0.950 sec)
INFO:tensorflow:global_step/sec: 104.891
INFO:tensorflow:loss = 0.32652405, step = 188500 (0.953 sec)
INFO:tensorflow:global_step/sec: 104.967
INFO:tensorflow:loss = 0.4057378, step = 188600 (0.952 sec)
INFO:tensorflow:global_step/sec: 105.663
INFO:tensorflow:loss = 0.35648105, step = 188700 (0.946 sec)
INFO:tensorflow:global_step/sec: 104.41
INFO:tensorflow:loss = 0.18178196, step = 188800 (0.958 sec)
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 188875...
INFO:tensorflow:Saving checkpoints for 188875 into MD_ave_weight_q_PhysProp/100_100_100/lr_1e-5/kfold_9/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 188875...
INFO:tensorfl

INFO:tensorflow:global_step/sec: 104.434
INFO:tensorflow:loss = 0.29561588, step = 194800 (0.957 sec)
INFO:tensorflow:global_step/sec: 104.798
INFO:tensorflow:loss = 0.2858804, step = 194900 (0.954 sec)
INFO:tensorflow:global_step/sec: 104.418
INFO:tensorflow:loss = 0.38820145, step = 195000 (0.957 sec)
INFO:tensorflow:global_step/sec: 104.78
INFO:tensorflow:loss = 0.2600602, step = 195100 (0.955 sec)
INFO:tensorflow:global_step/sec: 105.596
INFO:tensorflow:loss = 0.27778518, step = 195200 (0.947 sec)
INFO:tensorflow:global_step/sec: 104.786
INFO:tensorflow:loss = 0.2668317, step = 195300 (0.955 sec)
INFO:tensorflow:global_step/sec: 105.125
INFO:tensorflow:loss = 0.32957557, step = 195400 (0.951 sec)
INFO:tensorflow:global_step/sec: 104.21
INFO:tensorflow:loss = 0.20697282, step = 195500 (0.959 sec)
INFO:tensorflow:global_step/sec: 104.712
INFO:tensorflow:loss = 0.20452389, step = 195600 (0.955 sec)
INFO:tensorflow:global_step/sec: 105.639
INFO:tensorflow:loss = 0.19744891, step = 1957

In [8]:
# average the results
logps_martel = np.array(logps_martel)
logps_martel_ave = np.mean(logps_martel,axis=0)
#logps_martel_ave

logps_star = np.array(logps_star)
logps_star_ave = np.mean(logps_star,axis=0)

logps_nonstar = np.array(logps_nonstar)
logps_nonstar_ave = np.mean(logps_nonstar, axis=0)

In [9]:
# get the true value of martel
logp_exp_martel  = []
for x in results_martel:
    logp_exp_martel.append(x[3])

# get the true value of star
logp_exp_star = []
for x in results_star:
    logp_exp_star.append(x[3])
    
# get the true value of nonstar
logp_exp_nonstar = []
for x in results_nonstar:
    logp_exp_nonstar.append(x[3])

In [10]:
# calculate the mae, mse, and rmse of martel
mae_martel = mean_absolute_error(logps_martel_ave,logp_exp_martel)
mse_martel = mean_squared_error(logps_martel_ave,logp_exp_martel)
mae_martel,mse_martel,np.sqrt(mse_martel)

(0.810750931143091, 1.1553602851617193, 1.0748768697677513)

In [11]:
# calculate the mae, mse, and rmse of star
mae_star = mean_absolute_error(logps_star_ave,logp_exp_star)
mse_star = mean_squared_error(logps_star_ave,logp_exp_star)
mae_star,mse_star,np.sqrt(mse_star)

(0.6280671945997538, 0.701421690343194, 0.8375092180646098)

In [12]:
# calculate the mae, mse, and rmse of nonstar
mae_nonstar = mean_absolute_error(logps_nonstar_ave,logp_exp_nonstar)
mse_nonstar = mean_squared_error(logps_nonstar_ave,logp_exp_nonstar)
mae_nonstar,mse_nonstar,np.sqrt(mse_nonstar)

(0.34981019143506953, 0.258392444272231, 0.5083231691278994)